In [101]:
from PIL import Image
import numpy as np
import os
import pandas as pd

import glob
import argparse
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import oggm
from oggm import utils

from __future__ import print_function, division   # Ensures Python3 printing & division standard
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.pylab import rcParams

import numpy as np
import pandas as pd 
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score,mean_squared_error, mean_absolute_error, r2_score, roc_curve,auc
from sklearn.tree import DecisionTreeClassifier, plot_tree,export_graphviz
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.inspection import permutation_importance
from sklearn.neural_network import MLPClassifier

from scipy.stats import randint, poisson
from sklearn.preprocessing import MinMaxScaler

import xgboost as xgb
from xgboost import plot_tree

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta


from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

from ipywidgets import interactive
from graphviz import Source
from IPython.display import SVG

import csv

%matplotlib qt 

In [175]:
# Load the data
metadata_file = "metadata19_hmineq0.0_tmin20050000_mean_grid_20.csv"
glathida_rgis = pd.read_csv(metadata_file, low_memory=False)

# Group by 'RGIId' and get the unique 'Form' value for each 'RGIId'
unique_forms = glathida_rgis.groupby('RGIId')['Form'].first().reset_index()

# Function to convert 'Form' value to 2D vector
def convert_to_2d_vector(form_value):
    return [1, 0, 0] if form_value == 1 else [0, 1, 0]

# Apply the conversion function
unique_forms['Form_Vector'] = unique_forms['Form'].apply(convert_to_2d_vector)
form_vectors_array = np.array(unique_forms['Form_Vector'].tolist())

# Display the resulting DataFrame
print(unique_forms[['RGIId', 'Form_Vector']])
unique_forms['Form_Vector']

               RGIId Form_Vector
0     RGI60-01.08989   [0, 1, 0]
1     RGI60-01.10006   [0, 1, 0]
2     RGI60-01.10196   [0, 1, 0]
3     RGI60-01.10557   [0, 1, 0]
4     RGI60-01.10575   [0, 1, 0]
...              ...         ...
2316  RGI60-19.01298   [0, 1, 0]
2317  RGI60-19.01406   [0, 1, 0]
2318  RGI60-19.01407   [0, 1, 0]
2319  RGI60-19.01415   [0, 1, 0]
2320  RGI60-19.01721   [0, 1, 0]

[2321 rows x 2 columns]


0       [0, 1, 0]
1       [0, 1, 0]
2       [0, 1, 0]
3       [0, 1, 0]
4       [0, 1, 0]
          ...    
2316    [0, 1, 0]
2317    [0, 1, 0]
2318    [0, 1, 0]
2319    [0, 1, 0]
2320    [0, 1, 0]
Name: Form_Vector, Length: 2321, dtype: object

In [177]:
def vectorize_image(image_path, threshold=128):
    """
    Reads an image and vectorizes it to 0 and 1 depending on the color.

    Parameters:
    - image_path: str, path to the image file

    Returns:
    - binary_vector: numpy array of 0s and 1s
    """
    # Open the image and convert to RGB
    image = Image.open(image_path).convert('RGB')
    
    # Convert the image to a numpy array
    image_array = np.array(image)
    
    # Define a blue color range
    lower_blue = np.array([0, 0, 128])
    upper_blue = np.array([127, 127, 255])
    
    # Create a mask for blue pixels
    blue_mask = np.all((image_array >= lower_blue) & (image_array <= upper_blue), axis=-1)
    
    # Create a binary vector where blue is 1 and everything else is 0
    binary_vector = blue_mask.astype(int)
    
    return binary_vector

In [179]:
def process_folder(folder_path):
    """
    Processes all images in the specified folder, vectorizing each one.
    
    Parameters:
    - folder_path: str, path to the folder containing images

    Returns:
    - result_dict: dictionary, keys are image filenames and values are their binary vectors
    """
    result_dict = {}
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(folder_path, filename)
            binary_image = vectorize_image(image_path)
            result_dict[filename] = binary_image
    
    return result_dict

In [181]:
# Example usage
folder_path = r'C:\Users\damsg\OGGM\glacier_geometries_images'  # Replace with your folder path
processed_images = process_folder(folder_path)

def resize_array(array, size=(160, 160)):
    # Convert the array to PIL Image
    image = Image.fromarray(np.array(array, dtype='uint8'))
    # Resize the image
    resized_image = image.resize(size, resample=Image.BILINEAR)
    # Convert the resized image back to numpy array
    return np.array(resized_image)

# Assuming 'data_dict' is your dictionary containing multiple arrays
resized_arrays = [resize_array(array) for array in processed_images.values()]

# Stack all resized arrays into a single numpy array
resized_data_array = np.stack(resized_arrays, axis=0)


In [183]:
# processed_images.keys()
# #processed_images['RGI60-01.08989.png']

# # Idiotset = [processed_images['RGI60-01.08989.png'], processed_images['RGI60-01.10006.png'], processed_images['RGI60-01.10196.png'], processed_images['RGI60-01.10557.png'], processed_images['RGI60-01.10575.png'], processed_images['RGI60-01.10612.png'], processed_images['RGI60-01.10621.png'], processed_images['RGI60-01.10655.png'], processed_images['RGI60-01.10689.png'], processed_images['RGI60-01.10778.png']]
# Idiotset = []
# Idiotlabel = [0,0,0,0,2,1,2,1,1,0]
# # print(len(Idiotlabel))
# # processed_images.keys()


In [184]:
img_rows, img_cols = 160, 160
model = Sequential()

model.add(layers.Conv2D(filters=32,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))

model.add(layers.MaxPooling2D(pool_size=2, strides=None))

model.add(layers.Conv2D(filters=64,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 activation='relu'))

model.add(layers.MaxPooling2D(pool_size=2, strides=None))

model.add(layers.Dropout(rate=0.40))

model.add(layers.Flatten())

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=num_classes, activation='softmax'))

model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_60 (Conv2D)          (None, 160, 160, 32)      320       
                                                                 
 max_pooling2d_60 (MaxPoolin  (None, 80, 80, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_61 (Conv2D)          (None, 80, 80, 64)        18496     
                                                                 
 max_pooling2d_61 (MaxPoolin  (None, 40, 40, 64)       0         
 g2D)                                                            
                                                                 
 dropout_15 (Dropout)        (None, 40, 40, 64)        0         
                                                                 
 flatten_14 (Flatten)        (None, 102400)          

In [185]:
type(unique_forms['Form_Vector'])


pandas.core.series.Series

In [193]:

# np.array(unique_forms['Form_Vector'])
# resized_data_array = np.asarray(resized_data_array).astype(np.float32)
# resized_data_array = np.asarray(resized_data_array).astype(np.float32)
# y_test  = np.asarray(unique_forms['Form_Vector']).astype(np.float32)

model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['categorical_accuracy'])
model.fit(x = resized_data_array,y = form_vectors_array,epochs = 10)

Epoch 1/10
73/73 [==============================] - 21s 272ms/step - loss: 0.8548 - categorical_accuracy: 0.8014
Epoch 2/10
73/73 [==============================] - 21s 290ms/step - loss: 0.7350 - categorical_accuracy: 0.8014
Epoch 3/10
73/73 [==============================] - 21s 294ms/step - loss: 0.6571 - categorical_accuracy: 0.8014
Epoch 4/10
73/73 [==============================] - 21s 287ms/step - loss: 0.6097 - categorical_accuracy: 0.8014
Epoch 5/10
73/73 [==============================] - 21s 284ms/step - loss: 0.5831 - categorical_accuracy: 0.8014
Epoch 6/10
73/73 [==============================] - 21s 292ms/step - loss: 0.5663 - categorical_accuracy: 0.8014
Epoch 7/10
73/73 [==============================] - 21s 290ms/step - loss: 0.5538 - categorical_accuracy: 0.8014
Epoch 8/10
73/73 [==============================] - 22s 303ms/step - loss: 0.5459 - categorical_accuracy: 0.8014
Epoch 9/10
73/73 [==============================] - 26s 352ms/step - loss: 0.5389 - categorical_

In [156]:
np.array(unique_forms['Form_Vector'])

array([list([0, 1]), list([0, 1]), list([0, 1]), ..., list([0, 1]),
       list([0, 1]), list([0, 1])], dtype=object)

In [194]:
# y_test  = np.asarray(unique_forms['Form_Vector']).astype(np.float32)
# form_vectors_array = np.array(unique_forms['Form_Vector'].tolist())
print(np.shape(form_vectors_array))

(2321, 3)
